# Pseudo-Random Number Generation Algorithms

In [1]:
import numpy as np

## XOR Shift

Output from example C implementation 
from https://en.wikipedia.org/wiki/Xorshift 

In [2]:
test_values = [
    111,
    28437833,
    3204627429,
    3887305882,
    825450314,
    27031239,
    1742667205,
    3030794038,
    458757206,
    596234079,
    1445359534
]

In [3]:
def xorshift32(state):
    # Algorithm "xor" from p. 4 of Marsaglia, "Xorshift RNGs"
    state = (state ^ state << 13) & 0xffffffff;
    state = (state ^ state >> 17) & 0xffffffff;
    state = (state ^ state << 5) & 0xffffffff;
    return state

assert list(map(xorshift32, test_values[:-1])) == test_values[1:]

In [8]:
# This is not reversible!

## Linear congruential generator

This algorithm is reversible

In [9]:
# From: https://stackoverflow.com/a/58867375/1609514

def LCG(a, c, m, seed):     
    xi = seed
    while True:
        xf = (a * xi + c) % m
        xi = xf
        yield xf

def random_sample(n, interval, seed = 20200420162000):
    lower, upper = interval[0], interval[1]
    sample = []
    glibc = LCG(1103515245, 12345, 2 ** 32, seed)       # parameters as in GNU C Library

    for i in range(n):
        observation = (upper - lower) * (next(glibc) / (2 ** 32 - 1)) + lower
        sample.append(int(observation))

    return sample

In [10]:
sample = random_sample(10, [30, 90])
sample

[54, 77, 41, 61, 74, 49, 65, 66, 86, 39]

## Reversible LCG

Based on this C++ code from https://github.com/bobbaluba/rlcg/blob/master/include/rlcg.hpp:

In [11]:
def isPowerOfTwo(x):
    return (x & (x - 1)) == 0

assert all(isPowerOfTwo(x) for x in [0, 1, 2, 8, 256])
assert not any(isPowerOfTwo(x) for x in [-1, 3, 9, 257])

In [37]:
# N = maximum unsigned long value + 1
N = 1 << 64

def extendedEuclidX(a, b):
    return 1 if b == 0 else extendedEuclidY(b, a - b * (a // b));

def extendedEuclidY(a, b):
    return 0 if b == 0 else (extendedEuclidX(b, a - b * (a // b)) - (a // b) 
                             * extendedEuclidY(b, a - b * (a // b))) & (N - 1)

a_inverse_stored = {
    (6364136223846793005, 9223372036854775808): 13877824140714322085
}

def get_a_inverse_value(a, b):
    """Returns a pre-calculated value of a_inverse if one
    exists otherwise calculates it using extendedEuclidX(a, b).
    """
    return a_inverse_stored[(a, b)] if (a, b) in a_inverse_stored else extendedEuclidX(a, b)


class ReversibleLCG:
    """Reversible Linear Congruential Generator
    
    A type of pseudo-random number generator that supports generating
    random numbers in two directions (forwards and backwards).
    """

    # modulus
    M = 1 << 63

    # multiplicand
    A = 6364136223846793005

    # increment C
    C = 1442695040888963407

    # least significant bits to discard
    D = 32

    assert isPowerOfTwo(M), "M must be a power of two"

    def __init__(self, seed):
        self.x = seed
        self.ainverse = get_a_inverse_value(self.A, self.M)
        self.forward = True
    
    def __iter__(self):
        return self

    def __next__(self):
        """Compute and return next value in sequence."""
        if self.forward:
            return self.next()
        else:
            return self.prev()
    
    def reverse(self):
        self.forward = not self.forward
        
    def next(self):
        """Compute and return next value in sequence."""
        # nextx = (a * x + c) % m"""
        self.x = (self.A * self.x + self.C) & (self.M - 1)
        return self.x >> self.D
    
    def prev(self):
        """Compute and return value in sample."""
        # prevx = (ainverse * (x - c)) mod m
        self.x = self.ainverse * (self.x - self.C) & (self.M - 1)
        return self.x >> self.D;
    
    def max():
        return (self.M - 1) >> self.D


In [42]:
rng = ReversibleLCG(42)
test_values = [0, 293047021, 968358053, 1773127077]
assert [rng.next() for i in range(3)] == test_values[1:]
assert [rng.prev() for i in range(3)] == list(reversed(test_values[:-1]))

rng = ReversibleLCG(42)
[rng.next() for i in range(3)]

[121177411, 767878245, 153261767]

In [27]:
rng.forward

True

In [34]:
next(rng)

293047021

In [32]:
rng.forward = not rng.forward

## Bidirectional iterator

This was my answer to this stackoverflow question:
- https://stackoverflow.com/a/65536300/1609514

This is an example of a bi-directional iterator which simply counts up or down.

In [80]:
class BidirectionalCounter:
    """An iterator that can count in two directions (up
    and down).
    """

    def __init__(self, start):
        self.forward = True
        # Code to initialize the sequence
        self.x = start

    def __iter__(self):
        return self

    def __next__(self):
        if self.forward:
            return self.next()
        else:
            return self.prev()
    
    def reverse(self):
        self.forward = not self.forward
        
    def next(self):
        """Compute and return next value in sequence.
        """
        # Code to go forward
        self.x += 1
        return self.x
    
    def prev(self):
        """Compute and return previous value in sequence.
        """
        # Code to go backward
        self.x -= 1
        return self.x

In [81]:
my_counter = BidirectionalCounter(10)
print(next(my_counter))
print(next(my_counter))
my_counter.reverse()
print(next(my_counter))
print(next(my_counter))

11
12
11
10


In [82]:
from itertools import repeat

my_counter = BidirectionalCounter(0)
print([next(my_counter) for _ in range(10)])
my_counter.forward = False
print([next(my_counter) for _ in range(10)])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


In [83]:
my_counter = BidirectionalCounter(100)
np.fromiter(my_counter, dtype=int, count=10)

array([101, 102, 103, 104, 105, 106, 107, 108, 109, 110])